# 测试用文件

* 得到标签字典

In [ ]:
import pickle

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='latin1')
    return dict

dict = unpickle('dataset/cifar-100-python/meta')
# print(dict)

label_list = dict['fine_label_names']
# print(len(label_list))
# print(label_list)

* #取第一个图片显示一下

In [ ]:
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

import dataset

images = None
labels = None

for batch in dataset.train_loader:
    images, labels = batch
    break

label = labels[0]
image = images[0]

print("标签:" + str(label_list[label]))

#反归一化
mean = (0.5071, 0.4867, 0.4408)
std = (0.2675, 0.2565, 0.2761)

dmean = [-mean/std for mean, std in zip(mean, std)]
dstd = [1/std for std in std]

image = transforms.Normalize(dmean, dstd)(image)
image = transforms.ToPILImage(mode="RGB")(image)

plt.imshow(image)

* 测试tqdm

In [ ]:
from tqdm import tqdm

for i in tqdm(range(5)):
    for j in tqdm(range(5)):
        print(i," ",j)

* 测试numpy

In [ ]:
import numpy

matrix = [[1,2,3],[4,5,6]]
print(matrix[1][2])
print(matrix)
tensor = numpy.array(matrix)
print(tensor)
print(tensor[1][2])
print(tensor.shape)

* 形状测试

In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch

mean = (0.5071, 0.4867, 0.4408)
std = (0.2675, 0.2565, 0.2761)

transform=transforms.Compose([        
                    transforms.ToTensor(),
                    transforms.Normalize(mean=mean,std=std)
                ])

train_dataset=datasets.CIFAR100(
                    root='dataset',  
                    train=True,     
                    download=True,  
                    transform=transform
                )

train_loader = torch.utils.data.DataLoader(train_dataset, 
                                                        batch_size=16,
                                                        shuffle=True)

for data in train_loader:
    images, labels = data
    print(images.shape)
    print(images.dtype)
    print(labels.shape)
    print(labels[0])
    break

* 测试deepcopy会不会将权重一并复制
答案：会

In [ ]:
import copy

import torchvision

model = torchvision.models.resnet50(weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V2)
state_dict = model.state_dict()

new = copy.deepcopy(model)
new_dict = new.state_dict()

flag=True
for name, param in new_dict.items():
    if (state_dict[name] != param).any():
        flag = False
    print(param.dtype)

print(1 if flag else 0)



* 测试数据类型

In [ ]:
a = 1
b = 0.0
c = 2
d = a/c
print(type(a), type(b), type(c), type(d) )

import torch
a = torch.tensor([1,2,3])
b = torch.zeros_like(a, dtype=torch.float32)
print(b.dtype)

* 测试

In [ ]:
import torchvision
import torch

model = torchvision.models.resnet50(weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V2 )
new = torchvision.models.resnet50()

diff = {}
for name, data in model.state_dict().items():
    diff[name] = (data - new.state_dict()[name])

for name, data in diff.items():
    print(data.dtype)

In [ ]:
import torch

list = range(10)
a,b = torch.utils.data.random_split(list, [3,5])
for i in a:
    print(i)
print()
for i in b:
    print(i)

In [ ]:
import torch
import torchvision

model = torchvision.models.resnet50(weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V2)

new = torchvision.models.resnet50(weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V1)

diff = {}
for name, data in model.state_dict().items():
    diff[name] = (data - new.state_dict()[name])
for name, data in diff.items():
    print(data.dtype)
    break

weight_accumulator = {}
for name, data in model.state_dict().items():
    weight_accumulator[name] = torch.zeros_like(data, dtype=torch.float32)
for name, data in weight_accumulator.items():
    print(data.dtype)
    break

for name, _ in model.state_dict().items():
    weight_accumulator[name].add_(diff[name])
for name, data in weight_accumulator.items():
    print(data.dtype)
    break

for name, param in model.state_dict().items():
    # 得到每一层的更新
    update_per_layer = weight_accumulator[name] * 0.1
    print('---')
    print(update_per_layer.dtype)
    #   加上每一层更新
    # .add_()方法是不产生新张量
    param.add_(update_per_layer)
for name, data in model.state_dict().items():
    print(data.dtype)
    break

* 测试subsetrandomsampler

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

dataset = TensorDataset(torch.tensor(list(range(20))))  # 构造一个数据集（0到19）

idx = list(range(len(dataset)))  # 创建索引，SubsetRandomSampler会自动乱序

train_sampler = SubsetRandomSampler(idx[10:])  # 随机取80%的数据做训练集
test_sampler = SubsetRandomSampler(idx[:10])  # 随机取20%的数据做测试集

train_loader = DataLoader(dataset, sampler=train_sampler)
test_loader = DataLoader(dataset, sampler=test_sampler)

print('data for training:')
for i in train_loader:
    print(i)
print('data for testing:')
for i in test_loader:
    print(i)

In [6]:
import dataset
import util

util.dir_process()

train_set, val_set = dataset.get_dataset("cifar-10")

print(train_set)
print(train_set.classes)
print(train_set.class_to_idx)
print(train_set.tgz_md5)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: dataset
    Split: Train
    StandardTransform
Transform: Compose(
               RandomCrop(size=(32, 32), padding=4)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.201])
           )
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
c58f30108f718f92721af3b95e74349a


In [12]:
print(train_set.data.shape)
print(type(train_set[0][0]))
print(type(train_set[0][1]))


(50000, 32, 32, 3)
<class 'torch.Tensor'>
<class 'int'>
